In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import nnls
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("./carros_training.csv", index_col=0).reset_index(drop=True)
print(df.shape)
df.head()

(7440, 106)


,chevrolet_aveo,chevrolet_beat,chevrolet_camaro,chevrolet_captiva sport,chevrolet_cavalier,chevrolet_chevy,chevrolet_cheyenne,chevrolet_colorado,chevrolet_corsa,chevrolet_cruze,...,volkswagen_saveiro,volkswagen_teramont,volkswagen_tiguan,volkswagen_touareg,volkswagen_transporter,volkswagen_up!,volkswagen_vento,y_old,km,price
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,123467,31500
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,20,140000,33000
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,219143,35000
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,20,100000,38000
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,155000,39000


In [3]:
df.describe()

,chevrolet_aveo,chevrolet_beat,chevrolet_camaro,chevrolet_captiva sport,chevrolet_cavalier,chevrolet_chevy,chevrolet_cheyenne,chevrolet_colorado,chevrolet_corsa,chevrolet_cruze,...,volkswagen_saveiro,volkswagen_teramont,volkswagen_tiguan,volkswagen_touareg,volkswagen_transporter,volkswagen_up!,volkswagen_vento,y_old,km,price
count,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,...,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,7440.000000,7.440000e+03
mean,0.048118,0.010484,0.002957,0.004435,0.008602,0.004435,0.002823,0.002016,0.001747,0.014919,...,0.002151,0.003091,0.018548,0.002419,0.001613,0.004704,0.036962,4.692742,64181.727823,2.277767e+05
std,0.214031,0.101859,0.054301,0.066456,0.092354,0.066456,0.053056,0.044859,0.041767,0.121238,...,0.046327,0.055518,0.134933,0.049131,0.040131,0.068431,0.188682,3.295698,63255.620524,1.357168e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,3.200000e+03
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,29918.500000,1.450000e+05
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,54488.000000,1.980000e+05
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,84779.500000,2.700000e+05
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,20.000000,999999.000000,1.999000e+06


In [4]:
X = df.drop(['price'], axis=1)
Y = df.price
#Separar sets de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .10, random_state = 40)

In [5]:
#Evaluar resultados
def obtener_modelo(row):
    for col in row.index:
        if row[col] == 1:
            return col

def print_metrics(row, anunciado, prediccion):
    carro = obtener_modelo(row)
    err = abs(prediccion - anunciado)/anunciado
    print(f"\nPara un {carro} con {row['km']}km y {row['y_old']} años de uso")
    print(f"Precio anunciado: {anunciado} Precio predicho: {prediccion}.")
    print(f"Error: {round(100*err,2)}%")
    return err

def evaluar_modelo(X_test, Y_test, modelo):
    errAcc = 0
    columnas = X_test.shape[0]
    for i in range(columnas):
        row = X_test.iloc[i]
        prediccion = int(np.dot(modelo, row))
        anunciado = Y_test.iloc[i]
        errAcc += print_metrics(row, anunciado, prediccion)
    print(f"Error promedio: {round(100*errAcc/columnas, 2)}")

def evaluar_sk(X_test, Y_test, predicted):
    columnas = X_test.shape[0]
    for i in range(columnas):
        row = X_test.iloc[i]
        prediccion = int(predicted[i])
        anunciado = Y_test.iloc[i]
        print_metrics(row, anunciado, prediccion)


In [6]:
regr = linear_model.LinearRegression()
regr.fit(X_train, Y_train)
predicted = regr.predict(X_test)

evaluar_sk(X_test, Y_test, predicted)
print("Score:",regr.score(X_test, Y_test))
print("Error promedio:", round(100*(1 - regr.score(X_test, Y_test)), 2))


Para un ford_focus con 71000km y 10 años de uso
Precio anunciado: 88000 Precio predicho: 74213.
Error: 15.67%

Para un volkswagen_vento con 16088km y 1 años de uso
Precio anunciado: 229000 Precio predicho: 221195.
Error: 3.41%

Para un nissan_x-trail con 28820km y 2 años de uso
Precio anunciado: 410000 Precio predicho: 340208.
Error: 17.02%

Para un nissan_pathfinder con 76600km y 4 años de uso
Precio anunciado: 330000 Precio predicho: 376899.
Error: 14.21%

Para un nissan_np300 con 90000km y 6 años de uso
Precio anunciado: 168000 Precio predicho: 178185.
Error: 6.06%

Para un honda_cr-v con 53275km y 4 años de uso
Precio anunciado: 325000 Precio predicho: 317196.
Error: 2.4%

Para un honda_civic con 87900km y 4 años de uso
Precio anunciado: 260000 Precio predicho: 265598.
Error: 2.15%

Para un chevrolet_aveo con 39874km y 2 años de uso
Precio anunciado: 160900 Precio predicho: 184094.
Error: 14.42%

Para un chevrolet_aveo con 56318km y 2 años de uso
Precio anunciado: 192000 Precio pr

Para un mazda_3 sedan con 4500km y 1 años de uso
Precio anunciado: 379900 Precio predicho: 306106.
Error: 19.42%

Para un volkswagen_amarok con 43000km y 5 años de uso
Precio anunciado: 310000 Precio predicho: 297427.
Error: 4.06%

Para un ford_escape con 8400km y 3 años de uso
Precio anunciado: 378000 Precio predicho: 318226.
Error: 15.81%

Para un nissan_versa con 11835km y 1 años de uso
Precio anunciado: 195000 Precio predicho: 233050.
Error: 19.51%

Para un chevrolet_sonic con 52000km y 4 años de uso
Precio anunciado: 130000 Precio predicho: 163762.
Error: 25.97%

Para un honda_cr-v con 126025km y 9 años de uso
Precio anunciado: 189000 Precio predicho: 191289.
Error: 1.21%

Para un chevrolet_aveo con 80000km y 8 años de uso
Precio anunciado: 95500 Precio predicho: 38762.
Error: 59.41%

Para un chevrolet_trax con 71843km y 4 años de uso
Precio anunciado: 259000 Precio predicho: 218118.
Error: 15.78%

Para un volkswagen_amarok con 20884km y 3 años de uso
Precio anunciado: 299500 Prec

Para un volkswagen_tiguan con 34415km y 1 años de uso
Precio anunciado: 419900 Precio predicho: 408746.
Error: 2.66%

Para un nissan_march con 23200km y 1 años de uso
Precio anunciado: 138900 Precio predicho: 206826.
Error: 48.9%

Para un ford_fiesta sedan con 69000km y 4 años de uso
Precio anunciado: 158000 Precio predicho: 170002.
Error: 7.6%

Para un chevrolet_colorado con 7500km y 1 años de uso
Precio anunciado: 595000 Precio predicho: 478611.
Error: 19.56%

Para un honda_cr-v con 65200km y 4 años de uso
Precio anunciado: 299000 Precio predicho: 315741.
Error: 5.6%

Para un chevrolet_aveo con 44326km y 3 años de uso
Precio anunciado: 177000 Precio predicho: 160145.
Error: 9.52%

Para un mazda_5 con 92300km y 10 años de uso
Precio anunciado: 116000 Precio predicho: 106025.
Error: 8.6%

Para un ford_fiesta sedan con 31000km y 3 años de uso
Precio anunciado: 176000 Precio predicho: 198045.
Error: 12.53%

Para un honda_hr-v con 20000km y 2 años de uso
Precio anunciado: 348900 Precio pr

Precio anunciado: 185000 Precio predicho: 262945.
Error: 42.13%

Para un chevrolet_cruze con 30787km y 3 años de uso
Precio anunciado: 217000 Precio predicho: 233100.
Error: 7.42%

Para un honda_cr-v con 107000km y 12 años de uso
Precio anunciado: 135000 Precio predicho: 123392.
Error: 8.6%

Para un volkswagen_tiguan con 15538km y 2 años de uso
Precio anunciado: 398000 Precio predicho: 387642.
Error: 2.6%

Para un ford_focus con 77000km y 7 años de uso
Precio anunciado: 125000 Precio predicho: 143700.
Error: 14.96%

Para un chevrolet_trax con 57200km y 2 años de uso
Precio anunciado: 295000 Precio predicho: 266717.
Error: 9.59%

Para un ford_focus con 21783km y 3 años de uso
Precio anunciado: 265000 Precio predicho: 244061.
Error: 7.9%

Para un chevrolet_malibu con 80000km y 11 años de uso
Precio anunciado: 105000 Precio predicho: 100015.
Error: 4.75%

Para un ford_f-150 con 95000km y 3 años de uso
Precio anunciado: 390000 Precio predicho: 384155.
Error: 1.5%

Para un nissan_versa con 


Para un volkswagen_jetta con 53000km y 4 años de uso
Precio anunciado: 185000 Precio predicho: 218911.
Error: 18.33%

Para un ford_escape con 31382km y 3 años de uso
Precio anunciado: 349999 Precio predicho: 315422.
Error: 9.88%

Para un nissan_versa con 90000km y 8 años de uso
Precio anunciado: 105500 Precio predicho: 59670.
Error: 43.44%

Para un mazda_cx-5 con 38463km y 2 años de uso
Precio anunciado: 375000 Precio predicho: 333897.
Error: 10.96%

Para un ford_explorer con 65000km y 3 años de uso
Precio anunciado: 489000 Precio predicho: 432448.
Error: 11.56%

Para un ford_explorer con 84446km y 6 años de uso
Precio anunciado: 329000 Precio predicho: 359857.
Error: 9.38%

Para un nissan_pathfinder con 49000km y 3 años de uso
Precio anunciado: 468000 Precio predicho: 403673.
Error: 13.75%

Para un nissan_versa con 34819km y 2 años de uso
Precio anunciado: 185000 Precio predicho: 206839.
Error: 11.8%

Para un volkswagen_jetta con 170000km y 9 años de uso
Precio anunciado: 96500 Preci

In [7]:
modelo = nnls(X_train, Y_train)[0]
evaluar_modelo(X_test, Y_test, modelo)


Para un ford_focus con 71000km y 10 años de uso
Precio anunciado: 88000 Precio predicho: 167848.
Error: 90.74%

Para un volkswagen_vento con 16088km y 1 años de uso
Precio anunciado: 229000 Precio predicho: 182530.
Error: 20.29%

Para un nissan_x-trail con 28820km y 2 años de uso
Precio anunciado: 410000 Precio predicho: 294917.
Error: 28.07%

Para un nissan_pathfinder con 76600km y 4 años de uso
Precio anunciado: 330000 Precio predicho: 367058.
Error: 11.23%

Para un nissan_np300 con 90000km y 6 años de uso
Precio anunciado: 168000 Precio predicho: 208134.
Error: 23.89%

Para un honda_cr-v con 53275km y 4 años de uso
Precio anunciado: 325000 Precio predicho: 281420.
Error: 13.41%

Para un honda_civic con 87900km y 4 años de uso
Precio anunciado: 260000 Precio predicho: 213073.
Error: 18.05%

Para un chevrolet_aveo con 39874km y 2 años de uso
Precio anunciado: 160900 Precio predicho: 134287.
Error: 16.54%

Para un chevrolet_aveo con 56318km y 2 años de uso
Precio anunciado: 192000 Pre


Para un volkswagen_vento con 45049km y 1 años de uso
Precio anunciado: 179800 Precio predicho: 182530.
Error: 1.52%

Para un chevrolet_suburban con 45100km y 3 años de uso
Precio anunciado: 625000 Precio predicho: 631402.
Error: 1.02%

Para un nissan_march con 23000km y 2 años de uso
Precio anunciado: 160000 Precio predicho: 146256.
Error: 8.59%

Para un chevrolet_colorado con 43491km y 4 años de uso
Precio anunciado: 222000 Precio predicho: 350458.
Error: 57.86%

Para un ford_transit con 77000km y 3 años de uso
Precio anunciado: 330000 Precio predicho: 388033.
Error: 17.59%

Para un ford_expedition con 128000km y 5 años de uso
Precio anunciado: 355000 Precio predicho: 386686.
Error: 8.93%

Para un ford_escape con 47250km y 4 años de uso
Precio anunciado: 270000 Precio predicho: 256461.
Error: 5.01%

Para un honda_civic con 141000km y 10 años de uso
Precio anunciado: 115000 Precio predicho: 213073.
Error: 85.28%

Para un chevrolet_aveo con 60000km y 10 años de uso
Precio anunciado: 75

Para un volkswagen_jetta con 28527km y 2 años de uso
Precio anunciado: 255000 Precio predicho: 224286.
Error: 12.04%

Para un ford_escape con 53212km y 4 años de uso
Precio anunciado: 298000 Precio predicho: 256461.
Error: 13.94%

Para un honda_fit con 43500km y 3 años de uso
Precio anunciado: 190000 Precio predicho: 182307.
Error: 4.05%

Para un honda_odyssey con 120000km y 10 años de uso
Precio anunciado: 169000 Precio predicho: 281929.
Error: 66.82%

Para un chevrolet_aveo con 33000km y 1 años de uso
Precio anunciado: 190000 Precio predicho: 134287.
Error: 29.32%

Para un ford_figo sedan con 31205km y 3 años de uso
Precio anunciado: 172900 Precio predicho: 163035.
Error: 5.71%

Para un nissan_versa con 49500km y 7 años de uso
Precio anunciado: 128000 Precio predicho: 159214.
Error: 24.39%

Para un nissan_versa con 42439km y 2 años de uso
Precio anunciado: 179900 Precio predicho: 159214.
Error: 11.5%

Para un volkswagen_gol sedan con 8502km y 2 años de uso
Precio anunciado: 188900 Pr

Para un volkswagen_tiguan con 40423km y 2 años de uso
Precio anunciado: 345000 Precio predicho: 365943.
Error: 6.07%

Para un volkswagen_golf con 45761km y 4 años de uso
Precio anunciado: 234900 Precio predicho: 274388.
Error: 16.81%

Para un volkswagen_vento con 38500km y 4 años de uso
Precio anunciado: 140000 Precio predicho: 182530.
Error: 30.38%

Para un mazda_3 hatchback con 32000km y 2 años de uso
Precio anunciado: 335000 Precio predicho: 233575.
Error: 30.28%

Para un nissan_sentra con 64702km y 4 años de uso
Precio anunciado: 175000 Precio predicho: 177683.
Error: 1.53%

Para un ford_focus con 77895km y 6 años de uso
Precio anunciado: 132000 Precio predicho: 167848.
Error: 27.16%

Para un volkswagen_bora con 200000km y 12 años de uso
Precio anunciado: 80000 Precio predicho: 111897.
Error: 39.87%

Para un volkswagen_vento con 18823km y 1 años de uso
Precio anunciado: 235000 Precio predicho: 182530.
Error: 22.33%

Para un nissan_versa con 48500km y 4 años de uso
Precio anunciado:

Para un volkswagen_jetta con 37000km y 4 años de uso
Precio anunciado: 193000 Precio predicho: 224286.
Error: 16.21%

Para un chevrolet_aveo con 39874km y 2 años de uso
Precio anunciado: 160900 Precio predicho: 134287.
Error: 16.54%

Para un chevrolet_aveo con 71680km y 2 años de uso
Precio anunciado: 147000 Precio predicho: 134287.
Error: 8.65%

Para un volkswagen_polo hatchback con 15645km y 1 años de uso
Precio anunciado: 222000 Precio predicho: 202921.
Error: 8.59%

Para un mazda_cx-5 con 123000km y 6 años de uso
Precio anunciado: 220000 Precio predicho: 271531.
Error: 23.42%

Para un chevrolet_spark con 83000km y 5 años de uso
Precio anunciado: 89000 Precio predicho: 135269.
Error: 51.99%

Para un ford_lobo con 46000km y 2 años de uso
Precio anunciado: 740000 Precio predicho: 591748.
Error: 20.03%

Para un ford_escape con 55943km y 4 años de uso
Precio anunciado: 315000 Precio predicho: 256461.
Error: 18.58%

Para un chevrolet_malibu con 9000km y 4 años de uso
Precio anunciado: 34

In [8]:
pesos = []

for col in X_train:
    if col in 'km,y_old':
        continue
    indices = X_train[col] == 1
    dfX = X_train[indices][[col, 'km', 'y_old']]
    dfY = Y_train[indices]
    m = nnls(dfX, dfY)[0]
    pesos.append(m[0])

for elem in modelo[-2:]:
    pesos.append(elem)

modelo_chucky = np.array(pesos)
evaluar_modelo(X_test, Y_test, modelo_chucky)



Para un ford_focus con 71000km y 10 años de uso
Precio anunciado: 88000 Precio predicho: 167848.
Error: 90.74%

Para un volkswagen_vento con 16088km y 1 años de uso
Precio anunciado: 229000 Precio predicho: 182530.
Error: 20.29%

Para un nissan_x-trail con 28820km y 2 años de uso
Precio anunciado: 410000 Precio predicho: 294917.
Error: 28.07%

Para un nissan_pathfinder con 76600km y 4 años de uso
Precio anunciado: 330000 Precio predicho: 367058.
Error: 11.23%

Para un nissan_np300 con 90000km y 6 años de uso
Precio anunciado: 168000 Precio predicho: 208134.
Error: 23.89%

Para un honda_cr-v con 53275km y 4 años de uso
Precio anunciado: 325000 Precio predicho: 281420.
Error: 13.41%

Para un honda_civic con 87900km y 4 años de uso
Precio anunciado: 260000 Precio predicho: 213073.
Error: 18.05%

Para un chevrolet_aveo con 39874km y 2 años de uso
Precio anunciado: 160900 Precio predicho: 134287.
Error: 16.54%

Para un chevrolet_aveo con 56318km y 2 años de uso
Precio anunciado: 192000 Pre

Para un volkswagen_golf gti con 27109km y 2 años de uso
Precio anunciado: 440000 Precio predicho: 351134.
Error: 20.2%

Para un volkswagen_vento con 30000km y 1 años de uso
Precio anunciado: 209500 Precio predicho: 182530.
Error: 12.87%

Para un chevrolet_cavalier con 40046km y 1 años de uso
Precio anunciado: 209900 Precio predicho: 226075.
Error: 7.71%

Para un ford_fiesta hatchback con 40700km y 4 años de uso
Precio anunciado: 185000 Precio predicho: 134456.
Error: 27.32%

Para un volkswagen_vento con 45000km y 5 años de uso
Precio anunciado: 140000 Precio predicho: 182530.
Error: 30.38%

Para un volkswagen_jetta con 73000km y 6 años de uso
Precio anunciado: 160000 Precio predicho: 224286.
Error: 40.18%

Para un honda_odyssey con 16390km y 1 años de uso
Precio anunciado: 750000 Precio predicho: 281929.
Error: 62.41%

Para un nissan_np300 frontier con 45256km y 2 años de uso
Precio anunciado: 290000 Precio predicho: 299799.
Error: 3.38%

Para un ford_escape con 43700km y 2 años de uso

Error: 3.48%

Para un honda_br-v con 12300km y 2 años de uso
Precio anunciado: 315000 Precio predicho: 300806.
Error: 4.51%

Para un volkswagen_bora con 85000km y 10 años de uso
Precio anunciado: 88500 Precio predicho: 111897.
Error: 26.44%

Para un ford_expedition con 58000km y 3 años de uso
Precio anunciado: 550000 Precio predicho: 386686.
Error: 29.69%

Para un honda_accord con 130000km y 7 años de uso
Precio anunciado: 180000 Precio predicho: 242852.
Error: 34.92%

Para un volkswagen_jetta con 26890km y 3 años de uso
Precio anunciado: 210000 Precio predicho: 224286.
Error: 6.8%

Para un mazda_3 sedan con 38000km y 5 años de uso
Precio anunciado: 200000 Precio predicho: 186407.
Error: 6.8%

Para un chevrolet_captiva sport con 2010km y 10 años de uso
Precio anunciado: 110000 Precio predicho: 156791.
Error: 42.54%

Para un chevrolet_equinox con 30167km y 3 años de uso
Precio anunciado: 275000 Precio predicho: 294122.
Error: 6.95%

Para un chevrolet_equinox con 64378km y 4 años de uso


Para un nissan_versa con 66000km y 5 años de uso
Precio anunciado: 150000 Precio predicho: 159214.
Error: 6.14%

Para un volkswagen_polo hatchback con 752km y 1 años de uso
Precio anunciado: 215000 Precio predicho: 202921.
Error: 5.62%

Para un volkswagen_jetta con 108000km y 5 años de uso
Precio anunciado: 215000 Precio predicho: 224286.
Error: 4.32%

Para un nissan_march con 35000km y 4 años de uso
Precio anunciado: 138000 Precio predicho: 146256.
Error: 5.98%

Para un volkswagen_vento con 70967km y 3 años de uso
Precio anunciado: 179000 Precio predicho: 182530.
Error: 1.97%

Para un mazda_3 sedan con 39000km y 8 años de uso
Precio anunciado: 125000 Precio predicho: 186407.
Error: 49.13%

Para un honda_fit con 85956km y 5 años de uso
Precio anunciado: 139000 Precio predicho: 182307.
Error: 31.16%

Para un nissan_kicks con 90000km y 3 años de uso
Precio anunciado: 220000 Precio predicho: 268472.
Error: 22.03%

Para un volkswagen_clasico con 61243km y 5 años de uso
Precio anunciado: 13

Para un ford_explorer con 65000km y 3 años de uso
Precio anunciado: 489000 Precio predicho: 346996.
Error: 29.04%

Para un ford_explorer con 84446km y 6 años de uso
Precio anunciado: 329000 Precio predicho: 346996.
Error: 5.47%

Para un nissan_pathfinder con 49000km y 3 años de uso
Precio anunciado: 468000 Precio predicho: 367058.
Error: 21.57%

Para un nissan_versa con 34819km y 2 años de uso
Precio anunciado: 185000 Precio predicho: 159214.
Error: 13.94%

Para un volkswagen_jetta con 170000km y 9 años de uso
Precio anunciado: 96500 Precio predicho: 224286.
Error: 132.42%

Para un volkswagen_golf con 13000km y 2 años de uso
Precio anunciado: 340000 Precio predicho: 274388.
Error: 19.3%

Para un ford_edge con 185014km y 7 años de uso
Precio anunciado: 165000 Precio predicho: 347793.
Error: 110.78%

Para un nissan_altima con 72644km y 11 años de uso
Precio anunciado: 125000 Precio predicho: 205725.
Error: 64.58%

Para un mazda_3 hatchback con 110000km y 9 años de uso
Precio anunciado: 1

In [9]:
#sgd = linear_model.SGDRegressor(fit_intercept=False, verbose=1)
#sgd.fit(X_train, Y_train)
#predicted = sgd.predict(X_test)

#evaluar_sk(X_test, Y_test, predicted)
#print("Score:",sgd.score(X_test, Y_test))
#print("Error promedio:", round(100*(1 - sgd.score(X_test, Y_test)), 2))

In [10]:
"""percep = linear_model.Perceptron(fit_intercept=False, verbose=1, max_iter=1000, n_jobs=1)
percep.fit(X_train, Y_train)"""

'percep = linear_model.Perceptron(fit_intercept=False, verbose=1, max_iter=1000, n_jobs=1)\npercep.fit(X_train, Y_train)'

In [11]:
"""predicted = percep.predict(X_test)
print("score:",percep.score(X_test, Y_test))
print(predicted)"""

'predicted = percep.predict(X_test)\nprint("score:",percep.score(X_test, Y_test))\nprint(predicted)'